In [37]:
%pylab inline
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
import utm
from scipy.spatial import KDTree
jtplot.style(jtplot.infer_theme(), context='paper', fscale=2)
jtplot.figsize(x=20, y=12)

SHAPEFILES = glob.glob('../data/**/**/*.shp')
STREAMFLOW_META = './full_site_test_dataset.csv'

Populating the interactive namespace from numpy and matplotlib


In [2]:
def calcLatLon(northing, easting):
    from math import asin, atan2, cos, log, pow, sin, sqrt
 
    # CONSUS Albers variables (EPSG: 5070)
    RE_NAD83 = 6378137.0
    E_NAD83 = 0.0818187034 #Eccentricity
    D2R = 0.01745329251 #Pi/180
    standardParallel1 = 43.
    standardParallel2 = 47.
    centralMeridian = -114.
    originLat = 30
    originLon = 0
 
    m1 = cos(standardParallel1 * D2R)/sqrt(1.0 - pow((E_NAD83 * sin(standardParallel1 * D2R)), 2.0))
    m2 = cos(standardParallel2 * D2R)/sqrt(1.0 - pow((E_NAD83 * sin(standardParallel2 * D2R)), 2.0))
 
    def calcPhi(i):
        sinPhi = sin(i * D2R)
        return (1.0 - pow(E_NAD83, 2.0)) * ((sinPhi/(1.0 - pow((E_NAD83 * sinPhi), 2.0))) - 1.0/(2.0 * E_NAD83) * log((1.0 - E_NAD83 * sinPhi)/(1.0 + E_NAD83 * sinPhi)))
 
    q0 = calcPhi(originLat)
    q1 = calcPhi(standardParallel1)
    q2 = calcPhi(standardParallel2)
    nc = (pow(m1, 2.0) - pow(m2, 2.0)) / (q2 - q1)
    C = pow(m1, 2.0) + nc * q1
    rho0 = RE_NAD83 * sqrt(C - nc * q0) / nc
    rho = sqrt(pow(easting, 2.0) + pow((rho0 - northing), 2.0))
    q = (C - pow((rho * nc / RE_NAD83), 2.0)) / nc
    beta = asin(q / (1.0 - log((1.0 - E_NAD83) / (1.0 + E_NAD83)) * (1.0 - pow(E_NAD83, 2.0))/(2.0 * E_NAD83)))
    a = 1.0 / 3.0 * pow(E_NAD83, 2.0) + 31.0 / 180.0 * pow(E_NAD83, 4.0) + 517.0 / 5040.0 * pow(E_NAD83, 6.0)
    b = 23.0/360.0 * pow(E_NAD83, 4.0) + 251.0 / 3780.0 * pow(E_NAD83, 6.0)
    c = 761.0/45360.0 * pow(E_NAD83, 6.0)
    theta = atan2(easting, (rho0 - northing))
 
    lat = (beta + a * sin(2.0 * beta) + b * sin(4.0 * beta) + c * sin(6.0 * beta))/D2R
    lon = centralMeridian + (theta / D2R) / nc
    coords = [lat, lon]
 
    return coords
    #return coords
 
if __name__ == '__main__':
    northing = 487729.001
    easting = -254190.943
    calcLatLon(northing, easting)

# The stream temperature dataset includes temperature projections for two climatalogical periods, the 2040s and the 2080s. There are a variety of modeling options, but we will select out:

* S39_2040DM - Future Maximum Weekly Maximum Temperature (MWMT or 7DADM) stream scenario based on global climate model ensemble average projected changes for the A1B warming trajectory in the 2040s (2030-2059). Future stream deltas within a NorWeST unit account for differential sensitivity among streams so that cold streams warm less than warm streams
* S41_2080DM -  Future Maximum Weekly Maximum Temperature (MWMT or 7DADM) stream scenario based on global climate model ensemble average projected changes for the A1B warming trajectory in the 2080s (2070-2099). Future stream deltas within a NorWeST unit account for differential sensitivity among streams so that cold streams warm less than warm streams

In [ ]:
dataframes = [gp.GeoDataFrame.from_file(shpfile) for shpfile in SHAPEFILES]
gdf = gp.GeoDataFrame(pd.concat(dataframes, ignore_index=True))

In [88]:
# Extract out the variables we want to use because it's a large dataset
# and a smaller sample will be faster to work with
gdf_selected_columns = gdf[['S39_2040DM', 'S41_2080DM', 'geometry']]

translating_temperature_keys_dictionary = {'S39_2040DM': 'Stream Temperature 2040s',
                                         'S41_2080DM':  'Stream Temperature 2080s'}

In [126]:
# Remove the sites with NaNs
cleaned_up_gdf = gdf_selected_columns[gdf_selected_columns['S39_2040DM'].notnull()]

In [131]:
# Convert the coordinates from eastings/northings to degrees longitude
# and degrees latitude
lat_lons = []
for (i, point) in enumerate(cleaned_up_gdf.geometry[:]):
    # The false easting is from streamflow temperature
    # dataset documentation within the GIS shapefile
    false_easting = 1500000 
    northing = point.coords.xy[1][0]  
    easting = point.coords.xy[0][0] - false_easting
    [lat, lon] = calcLatLon(northing, easting)
    lat_lons.append([lat, lon])
temperature_sites = np.array(lat_lons)

In [132]:
streamflow_sites = pd.read_csv(STREAMFLOW_META)

# Select out the sites in the United States because the temperature data
# is only available in the U.S. So, south of the 49th parallel!
streamflow_sites = streamflow_sites[streamflow_sites['Latitude']<49]

In [133]:
collated_dataset = pd.DataFrame(index=streamflow_sites['Site ID'], 
                                columns=list(translating_temperature_keys_dictionary.values()))
for site in streamflow_sites['Site ID']:
    
# Loop through each location in the streamflow set and
# select the 10 nearest points within the stream temperature set
    point = [streamflow_sites[streamflow_sites['Site ID']==site]['Latitude'].values[0],
             streamflow_sites[streamflow_sites['Site ID']==site]['Longitude'].values[0]]
    gdf_selected_columns
    tree = KDTree(temperature_sites, leafsize=temperature_sites.shape[0]+1)
    distances, ndx = tree.query([point], k=10)
    nearest_neighbors_data = cleaned_up_gdf.iloc[list(ndx[0])]
    for variable in translating_temperature_keys_dictionary.keys():
        collated_dataset.set_value(site, 
                        translating_temperature_keys_dictionary[variable], 
                        nearest_neighbors_data[variable].mean())

/Users/orianachegwidden/anaconda/envs/futurefish/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [144]:
def get_model_ts(infilename, na_values='-9999', comment='#',
                 rename_columns=None, column='streamflow'):
    '''Retrieve modeled time series from ASCII file
    Parameters
    ----------
    infilename : str
        Pathname for file
    na_values : str, optional
        Values that should be converted to `NA`. Default value is '-9999'
    comment : str, optional
        Comment indicator at the start of the line. Default value is '#'=
    rename_columns: dict or None, optional
        Dictionary to rename columns. Default value is None
    column = str, optional
        Name of the column that will be returned. Default value is 'streamflow'
    Returns
    -------
    pandas.Series
        Column from file as a pandas.Series
    '''
    ts = pd.read_csv(infilename, comment=comment, na_values=na_values,
                     index_col=0, parse_dates=True)
    # renaming of columns may seem superfluous if we are converting to a Series
    # anyway, but it allows all the Series to have the same name
    if rename_columns:
        ts.columns = [column]
    return pd.Series(ts[column])

In [158]:
def metric_min7day_streamflow(df, future_slice):
    df_roll_mean = df.rolling(window=7, min_periods=7, center=False).mean()
    df_annual = df_roll_mean.resample('AS', how='min')[future_slice]
    return df_annual
#     da_annual.sel(time=future_slice).mean()    

In [167]:
streamflow_timeframes = {'Streamflow 2040s': slice('2029-10-01', '2059-09-30'),
                        'Streamflow 2080s': slice('2069-10-01', '2099-09-30')}
for site in streamflow_sites['Site ID']:
    streamflow_file = '/Users/orianachegwidden/Downloads/CCSM4_RCP85-streamflow-1.0/'+\
                    'streamflow/CCSM4_RCP85_MACA_VIC_P2-'+site+'-streamflow-1.0.csv'
    df = get_model_ts(streamflow_file)
    for (label, timeframe) in streamflow_timeframes.items():
        collated_dataset.set_value(site, label, 
                                   metric_min7day_streamflow(df, 
                                        timeframe).quantile(q=0.1))

/Users/orianachegwidden/anaconda/envs/futurefish/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).min()
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/orianachegwidden/anaconda/envs/futurefish/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


2042.1552285711189
2727.7714428578297
4217.675485714396
67905.5105142888
251.91715714284766
62201.72422857291
418.0073714285661
60651.47672857583
418.0073714285661
1868.5771714285572
65027.500657146935
223.4676428571607
177.39204285713018
98.91687142856749
15091.14704285649
11340.41702857103
879.1980571429212
276.927500000021
11410.01848571477
17669.9958285724
17282.382814285782
13570.969371428486
17560.393542856647
17201.752828571123
739.8535000000353
11359.611014285783
0.6388999999962345
1924.705214285914
354.65498571441594
31.33401428570806
1815.8234285709666
31469.977242861816
1298.9098999999583
31198.29708571471
455.85921428578354
36614.24044285686
35940.925257145565
34746.968014282465
36317.6241285686
32715.3678999981
2655.4331714285004
8662.42881428577
8937.151999999069
8712.34054285808
6331.774742857948
1023.0949857141553
223.47804285709373
2251.2019999999748
1865.3248714283875
6245.42369999987
381.254271428566
87.25799999999586
6115.984857143174
207.63647142852562
515.44381428

In [169]:
collated_dataset.to_csv('./sites_streamflow_stream_temperature.csv')

In [168]:
collated_dataset

,Stream Temperature 2040s,Stream Temperature 2080s,Streamflow 2040s,Streamflow 2080s
Site ID,,,,
ALB,24.304,25.621,2042.155229,1955.022943
LIB,18.852,19.938,2727.771443,2488.091671
BFE,12.079,12.824,4217.675486,3687.116386
BON,19.896,21.094,67905.510514,70655.700086
HOD,22.852,23.982,251.917157,250.765671
JDA,29.051,30.255,62201.724229,64692.853029
KLC,24.66,25.814,418.007371,557.918686
MCN,30.636,31.858,60651.476729,63727.254743
PEL,22.497,23.624,418.007371,557.918686
